In [1]:
# GPU setting
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# modules setting
from sklearn.metrics import confusion_matrix
import itertools
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import random
import cv2

/home/project/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# PREPROCESSED DATA DIR

In [4]:
# data selection
# version = ['pillcam_v1', 'pillcam_v2', 'pillcam_v3'] Only version 3
# cls = ['negative', 'hemorrhagic', 'depressed', 'protruded']
# lesion = {'negative' : ['negative'], 
#           'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding'],
#           'depressed': ['erosion', 'ulcer', 'stricture'],
#           'protruded': ['ampulla_of_vater', 'lymphoid_follicles', 'small_bowel_tumor']}
cls = ['negative', 'hemorrhagic']
lesion = {'negative' : ['negative'], 
          'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding']}

In [5]:
# path of data folder
dir_data_pre = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core/'
path_folder_pre = {}

for i in cls:
    path_folder_pre[i] = {}
    for j in lesion[i]:
        temp = dir_data_pre + '{0}_{1}/{2}'.format(cls.index(i), i, j)
        path_folder_pre[i][j] = temp
        
print(path_folder_pre)

{'negative': {'negative': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core/0_negative/negative'}, 'hemorrhagic': {'red_spot': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core/1_hemorrhagic/red_spot', 'angioectasia': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core/1_hemorrhagic/angioectasia', 'active_bleeding': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core/1_hemorrhagic/active_bleeding'}}


In [6]:
list_data_pre = {}
for i in cls:
    list_data_pre[i] = {}
    for j in lesion[i]:
        list_data_pre[i][j] = []
        temp = os.listdir(path_folder_pre[i][j])
        list_data_pre[i][j].append(temp)

print(list_data_pre)

{'negative': {'negative': [['명나___00-49-22___2008035.jpg', '왕신___04-28-48___2048286.jpg', 'PI___04-30-12___2045136.jpg', '승이___03-02-34___2035043.jpg', '원장___03-50-30___2044761.jpg', '정주___10-28-04___2092504.jpg', '명나___02-20-21___2022755.jpg', '명이___06-13-06___2061953.jpg', '옥정___03-30-20___2035316.jpg', '민방___00-09-48___2003052.jpg', '승이___02-29-07___2030626.jpg', '기김___00-15-05___1001811.jpg', '솔신___10-20-53___1074514.jpg', '민방___01-57-03___2031151.jpg', '계신___02-03-34___2024160.jpg', '사고___02-04-05___2026902.jpg', '순임___02-45-18___2024139.jpg', '경양___04-24-08___2048947.jpg', '강은___00-53-27___2009679.jpg', '복지___00-55-39___2011773.jpg', '윤김___06-43-43___2065547.jpg', '병김___02-36-11___2033307.jpg', '진강___00-43-25___2013987.jpg', '민강___01-42-13___2023440.jpg', '손정___05-14-37___2054870.jpg', '명이___06-29-18___2063897.jpg', '승한___02-30-58___1018115.jpg', '혜유___01-53-53___2021305.jpg', '장한___01-22-28___2022228.jpg', '상이___02-57-40___2034355.jpg', '양임___01-34-43___2022948.jpg', '기강___02-28

# Example

In [7]:
temp = cv2.imread('/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset_v3_only_pre/0_negative/negative/' + list_data_pre['negative']['negative'][0][0])
rows, cols = temp.shape[:2]

M = cv2.getRotationMatrix2D(center = (cols/2, rows/2), angle = 180, scale = 1)
temp_rotated = cv2.warpAffine(temp, M, dsize = (rows, cols))

temp_rgb = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
temp_rotated_rgb = cv2.cvtColor(temp_rotated, cv2.COLOR_BGR2RGB)

plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
plt.imshow(temp_rgb)
plt.subplot(1,2,2)
plt.imshow(temp_rotated_rgb)

AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
def rotate90(img, times):
    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D(center = (cols/2, rows/2), angle = 90*times, scale = 1)
    img_rotated = cv2.warpAffine(img, M, dsize = (rows, cols))
    return img_rotated

# Augmentation

In [9]:
# dir_data_pre = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset_v3_only_pre/'
dir_data_pre_aug = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core_aug/'
for i in cls:
    for j in lesion[i]:
        for k in list_data_pre[i][j][0]:
            temp = cv2.imread(dir_data_pre + '{0}_{1}/{2}/{3}'.format(cls.index(i), i, j, k)) # Original
            temp_r1 = rotate90(temp, 1) # 90도
            temp_r2 = rotate90(temp, 2) # 180도
            temp_r3 = rotate90(temp, 3) # 270도
            
            temp_f = np.flipud(temp) # flip up and down
            temp_f_r1 = rotate90(temp_f, 1) # 90도
            temp_f_r2 = rotate90(temp_f, 2) # 180도
            temp_f_r3 = rotate90(temp_f, 3) # 270도
            
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/{3}'.format(cls.index(i), i, j, k), temp)
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/r1_{3}'.format(cls.index(i), i, j, k), temp_r1)
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/r2_{3}'.format(cls.index(i), i, j, k), temp_r2)
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/r3_{3}'.format(cls.index(i), i, j, k), temp_r3)
            
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/f_{3}'.format(cls.index(i), i, j, k), temp_f)
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/f_r1_{3}'.format(cls.index(i), i, j, k), temp_f_r1)
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/f_r2_{3}'.format(cls.index(i), i, j, k), temp_f_r2)
            cv2.imwrite(dir_data_pre_aug + '{0}_{1}/{2}/f_r3_{3}'.format(cls.index(i), i, j, k), temp_f_r3)

# 확인

## - Original

In [10]:
print(list_data_pre)

{'negative': {'negative': [['명나___00-49-22___2008035.jpg', '왕신___04-28-48___2048286.jpg', 'PI___04-30-12___2045136.jpg', '승이___03-02-34___2035043.jpg', '원장___03-50-30___2044761.jpg', '정주___10-28-04___2092504.jpg', '명나___02-20-21___2022755.jpg', '명이___06-13-06___2061953.jpg', '옥정___03-30-20___2035316.jpg', '민방___00-09-48___2003052.jpg', '승이___02-29-07___2030626.jpg', '기김___00-15-05___1001811.jpg', '솔신___10-20-53___1074514.jpg', '민방___01-57-03___2031151.jpg', '계신___02-03-34___2024160.jpg', '사고___02-04-05___2026902.jpg', '순임___02-45-18___2024139.jpg', '경양___04-24-08___2048947.jpg', '강은___00-53-27___2009679.jpg', '복지___00-55-39___2011773.jpg', '윤김___06-43-43___2065547.jpg', '병김___02-36-11___2033307.jpg', '진강___00-43-25___2013987.jpg', '민강___01-42-13___2023440.jpg', '손정___05-14-37___2054870.jpg', '명이___06-29-18___2063897.jpg', '승한___02-30-58___1018115.jpg', '혜유___01-53-53___2021305.jpg', '장한___01-22-28___2022228.jpg', '상이___02-57-40___2034355.jpg', '양임___01-34-43___2022948.jpg', '기강___02-28

In [11]:
len(list_data_pre['negative']['negative'][0])

128

## - Augmentation

In [14]:
# path of data folder
dir_data_pre_aug = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core_aug/'
path_folder_pre_aug = {}

for i in cls:
    path_folder_pre_aug[i] = {}
    for j in lesion[i]:
        temp = dir_data_pre_aug + '{0}_{1}/{2}'.format(cls.index(i), i, j)
        path_folder_pre_aug[i][j] = temp
        
print(path_folder_pre_aug)

{'negative': {'negative': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core_aug/0_negative/negative'}, 'hemorrhagic': {'red_spot': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core_aug/1_hemorrhagic/red_spot', 'angioectasia': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core_aug/1_hemorrhagic/angioectasia', 'active_bleeding': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/dataset_v3_only_pre_core_aug/1_hemorrhagic/active_bleeding'}}


In [15]:
list_data_pre_aug = {}
for i in cls:
    list_data_pre_aug[i] = {}
    for j in lesion[i]:
        list_data_pre_aug[i][j] = []
        temp = os.listdir(path_folder_pre_aug[i][j])
        list_data_pre_aug[i][j].append(temp)

print(list_data_pre_aug)

{'negative': {'negative': [['r1_경양___04-24-08___2048947.jpg', 'f_강은___02-18-28___2020419.jpg', 'r3_윤김___06-08-33___2061327.jpg', 'r1_순하___08-56-32___2081488.jpg', 'f_r2_경양___05-11-36___2055065.jpg', 'f_r3_왕신___04-28-48___2048286.jpg', 'f_r3_순하___08-56-32___2081488.jpg', 'f_r1_한이___01-15-33___2021728.jpg', 'f_r1_성양___00-54-27___2012446.jpg', '명나___00-49-22___2008035.jpg', 'f_r1_윤김___04-36-10___2050238.jpg', 'r3_명이___03-10-23___2040021.jpg', '왕신___04-28-48___2048286.jpg', 'PI___04-30-12___2045136.jpg', 'f_r3_은임___02-29-41___2035091.jpg', 'f_r1_병김___02-36-11___2033307.jpg', 'f_r1_혜유___02-38-56___2028977.jpg', '승이___03-02-34___2035043.jpg', 'r2_민강___01-42-13___2023440.jpg', 'r2_서미___05-15-56___1037723.jpg', 'f_r1_명나___00-08-43___2002035.jpg', 'r1_기김___00-31-02___2009488.jpg', 'f_r3_청이___02-34-43___2027294.jpg', 'r1_승한___02-38-46___1019051.jpg', 'f_PI___02-07-28___2025907.jpg', 'f_솔신___09-11-41___1066208.jpg', 'r1_계신___02-03-34___2024160.jpg', 'r2_진남___04-50-16___2051925.jpg', 'f_양임___01-34

In [16]:
for i in cls:
    for j in lesion[i]:
        print(i + '\t' + j, len(list_data_pre_aug[i][j][0]))

negative	negative 1024
hemorrhagic	red_spot 632
hemorrhagic	angioectasia 112
hemorrhagic	active_bleeding 120
